In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time

In [ ]:
starry.config.lazy = False
starry.config.quiet = True

## Error analysis

In [ ]:
def get_error(ydeg=15, wta=30, **kwargs):

    # Instantiate
    map = starry.Map(ydeg, **kwargs)

    # Apply the differential rotation then undo it.
    # If the transform is one-to-one, this should yield
    # the identity matrix.
    wta = np.ones(map.Ny) * (wta * np.pi / 180)
    I = map.ops.tensordotD(
        map.ops.tensordotD(np.eye(map.Ny), wta, np.array(1.0)), -wta, np.array(1.0)
    )

    # Compute the mean difference between the diagonal and unity for each l
    x = np.abs(1 - np.diag(I))
    mu = np.array([np.mean(x[l ** 2 : (l + 1) ** 2]) for l in range(map.ydeg + 1)])

    return mu

### Error as a function of spherical harmonic degree

In [ ]:
wta = 30
error10 = get_error(ydeg=10, wta=wta)
error20 = get_error(ydeg=20, wta=wta)
error30 = get_error(ydeg=30, wta=wta)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(error10, color="C0", label=r"$l = 10$")
plt.plot(error20, color="C1", label=r"$l = 20$")
plt.plot(error30, color="C2", label=r"$l = 30$")
plt.grid()
plt.yscale("log")
plt.gca().set_yticks([1e-12, 1e-9, 1e-6, 1e-3, 1e0])
plt.legend(loc="lower right")
plt.title(r"$\omega t \alpha = 30^\circ$")
plt.xlabel("spherical harmonic degree")
plt.ylabel(r"relative error");

### Error as a function of $\omega t \alpha$

In [ ]:
ydeg = 20
wta = np.linspace(0, 180, 25)
error = np.zeros((len(wta), ydeg + 1))
for i, wta_i in tqdm(enumerate(wta), total=len(wta)):
    error[i] = get_error(ydeg=ydeg, wta=wta_i)

In [ ]:
plt.figure(figsize=(12, 6))
logerror = np.log10(error)
plt.imshow(
    logerror, extent=(0, ydeg, 0, 180), origin="lower", aspect="auto", vmin=-12, vmax=0
)
cbar = plt.colorbar()
cbar.set_ticks([-12, -9, -6, -3, 0])
cbar.set_ticklabels(
    [r"$10^{-12}$", r"$10^{-9}$", r"$10^{-6}$", r"$10^{-3}$", r"$10^{0}$"]
)
cbar.set_label("relative error")
cont = plt.contour(
    np.arange(ydeg + 1),
    wta,
    logerror,
    [-9, -6, -3, -2, -1],
    colors="w",
    linestyles="solid",
)
fmt = {}
strs = ["1 ppb", "1 ppm", "1 ppt", "1%", "10%"]
for l, s in zip(cont.levels, strs):
    fmt[l] = s
plt.clabel(cont, cont.levels, inline=True, fmt=fmt, fontsize=10)
plt.xlabel("spherical harmonic degree")
plt.ylabel(r"$\omega t \alpha$ [degrees]")
plt.gca().set_yticks([0, 30, 60, 90, 120, 150, 180])
plt.gca().set_xticks([0, 5, 10, 15, 20])
plt.gca().set_xticklabels(["0", "5", "10", "15", "20"]);

### Example

In [ ]:
theta = 90
ydeg = 20

map = starry.Map(ydeg)
map.load("earth", sigma=0.08)
map.amp = 1.0
map[ydeg - 5 :, :] = 0

# Original image
img0 = map.render(projection="rect")

# Differentially rotate it
map[:, :] = map.ops.tensordotD(
    map.y.reshape(1, -1), np.array(theta * np.pi / 180), np.array(1.0)
)
img1 = map.render(projection="rect")

# Undo the operation
map[:, :] = map.ops.tensordotD(
    map.y.reshape(1, -1), np.array(-theta * np.pi / 180), np.array(1.0)
)
img2 = map.render(projection="rect")


fig, ax = plt.subplots(2, 2, figsize=(12, 7))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
ax = ax.flatten()
for axis in ax:
    axis.set_xticks([])
    axis.set_yticks([])

im = ax[0].imshow(
    img0, origin="lower", extent=(-180, 180, -90, 90), cmap="plasma", vmin=0, vmax=1
)
cax = make_axes_locatable(ax[0]).append_axes("right", size="5%", pad=0.05)
cax.axis("off")
ax[0].set_title("original")

im = ax[1].imshow(
    img1, origin="lower", extent=(-180, 180, -90, 90), cmap="plasma", vmin=0, vmax=1
)
cax = make_axes_locatable(ax[1]).append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im, ax=ax[1], cax=cax, shrink=1)
ax[1].set_title("transformed")

im = ax[2].imshow(
    img2, origin="lower", extent=(-180, 180, -90, 90), cmap="plasma", vmin=0, vmax=1
)
cax = make_axes_locatable(ax[2]).append_axes("right", size="5%", pad=0.05)
cax.axis("off")
ax[2].set_title("reconstructed")

im = ax[3].imshow(
    img2 - img0,
    origin="lower",
    extent=(-180, 180, -90, 90),
    cmap="RdBu",
    vmin=-0.05,
    vmax=0.05,
)
cax = make_axes_locatable(ax[3]).append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im, ax=ax[3], cax=cax, shrink=1)
cbar.set_ticks([-0.05, -0.025, 0, 0.025, 0.05])
ax[3].set_title("relative error");

## Timing tests

In [ ]:
ydeg = 20
npts = 1000
ncalls = 10

theta = np.linspace(0, 360.0, npts)

t0 = np.zeros(ydeg + 1)
tD = np.zeros(ydeg + 1)
for d in tqdm(range(ydeg + 1)):
    map = starry.Map(ydeg=d)
    map.flux()  # force compile

    # Standard
    map.alpha = 0.0
    tstart = time.time()
    for k in range(ncalls):
        map.flux(theta=theta)
    t0[d] = (time.time() - tstart) / ncalls / npts

    # Differentially rotated
    map.alpha = 1.0
    tstart = time.time()
    for k in range(ncalls):
        map.flux(theta=theta)
    tD[d] = (time.time() - tstart) / ncalls / npts

In [ ]:
plt.plot(t0, label="solid")
plt.plot(tD, label="differential")

l = np.arange(5, ydeg + 1)
plt.plot(l, 1e-6 + 1e-9 * l ** 4, "k-", lw=3, ls="--", alpha=0.25, label=r"$l^4$")

plt.legend()
plt.yscale("log")
plt.gca().set_xticks([0, 5, 10, 15, 20])
plt.gca().set_xticklabels(["0", "5", "10", "15", "20"])
plt.ylabel("time [s]")
plt.xlabel("spherical harmonic degree");